# LS4 Image Reduction Tutorial

In [1]:
# in-built
import os, glob

# standard

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy import stats
from scipy.optimize import curve_fit

# specialty

from astropy.io import fits
import ls4reduce as ls4
#import sep

# magic
%matplotlib inline
%config InlineBackend.figure_format='retina'

## 1. Overscan subtraction

Suppose that we have a list of compressed FITS images (.fz) at a certain exposure time. To start reducing the image, use the overscan subtraction function first.

In [ ]:
darks0s = glob.glob('LS4Darks/darks_0s_*.fz') # make a list of the raw images at a single exposure time
for x in range(0,len(darks0s)):
    ls4.overscanSubtraction(darks0s[x], 2, 'LS4Darks')
    ls4.overscanSubtraction(darks0s[x], 3, 'LS4Darks')

This function takes the filelist at a single exposure time, the extension number, and an output directory.

## 2. Generate superbias

Next, use the makeSuperbias function to create the superbias frame for each extension, for each exposure time.

In [ ]:
darks0s_os_2 = glob.glob('LS4Darks/darks_0s_*2.fits')
darks0s_os_3 = glob.glob('LS4Darks/darks_0s_*3.fits')

ls4.makeSuperbias(darks0s_os_2, 'LS4Darks/superbias_2.fits')
ls4.makeSuperbias(darks0s_os_3, 'LS4Darks/superbias_3.fits')

This function takes the filelist at a single exposure time at a single extension and the destination of the superbias frame. You need to specify the filename with a .fits extension.

## 3. Subtract superbias

Subtract the respective superbias frame from each image that has been overscan subtracted.

In [ ]:
# Superbias for amps 2 and 3
superbias_2 = 'LS4Darks/superbias_2.fits'
superbias_3 = 'LS4Darks/superbias_3.fits'

ls4.biasSubtraction(darks0s_os_2, superbias_2)
ls4.biasSubtraction(darks0s_os_3, superbias_3)

This function takes the same filelist as the one for generating the superbias frames and the path to the superbias FITS image itself. 

To be implemented: make destination of output specifiable

## 4. Merge amps

Stitch together the readouts from the two amps by flipping the x-axis of extension 3 and combining with extension 2 using `mergeAmps`.

In [ ]:
ls4.mergeAmps(sorted(glob.glob('LS4Darks/bs_darks_0s_*2.fits')), sorted(glob.glob('LS4Darks/bs_darks_0s_*3.fits')))

This function takes a filelist of bias subtracted images at extension 2 and 3. The order of the filelist is important because they must correspond to each other for the merging of amps to correspond to its counterpart. Using `sorted` around the filelists is probably best here.

# Computing Gain

For x-ray exposed images, the first step to computing gain is extracting and computing the flux of the sources in the image. Use `computeFlux` to do these two steps.

In [ ]:
darks0s_fluxes = ls4.computeFlux(glob.glob('LS4Darks/merged*.fits'), 700, 5.0, False)

This function takes in a filelist of reduced merged FITS image files, a pixel value threshold from which to extract, a radius for doing photometry, and a boolean for whether a ds9 regions file is to be generated for convenience. The output is a list of 1D numpy arrays of extracted flux values.

In [ ]:
gain = ls4.computeGain(darks0s_fluxes, 800, (0,100000), True)

Using the `computeGain` function, we can take our list of 1D numpy arrays from `computeFlux` and generate a histogram fitted to a double Gaussian to compute the actual gain value. The second argument is the number of bins for the histogram, the third argument is the range of the histogram, and the boolean is for whether you desire to have output plots.

# Computing Read Noise

Use `computeRN` to calculate the read noise of a list of raw images. The raw iamges are used for computing the read noise because we are interested in taking the overscan region data and median subtracting within the overscan region.

In [ ]:
darks0s = np.array(['LS4Darks/darks_0s_2.fz'])
computeRN(darks0s, 51.01813625)

This function takes a filelist of raw compressed FITS images and a float of the gain. The output is two normalized histograms (one for each extension) of the median-subtracted overscan region pixels. This is a Gaussian distribution and is fitted with a Gaussian. The read noise is the $\sigma$ of the Gaussian divided by the gain.